# 神经网络训练技巧

| ![](../../_static/logo/megvii-32.svg) [在 MegStudio 运行](https://studio.brainpp.com/project/#) | ![](../../_static/logo/github-32.svg) [查看源文件](https://github.com/MegEngine/Documentation/blob/main/source/getting-started/beginner/neural-network-traning-tricks.ipynb) |
| --- | --- |

通过引入（卷积）神经网络模型，我们在 CIFAR-10 图片分类任务上已经能够达到 50% 左右的分类准确率。但是，探索从未停止！

在本次的教程中，我们会接触到神经网络模型训练过程中的常见技巧，并尝试对其原理做出一定的解释。

请先运行下面的代码，检验你的环境中是否已经安装好 MegEngine（[访问官网安装教程](https://megengine.org.cn/install)）：

In [8]:
import megengine

print(megengine.__version__)

1.4.0


我们需要回顾一下我们之前实现的 LeNet 模型训练过程，不过这次我们稍微整理了一下代码：

epoch = 0, loss = 0.034
epoch = 10, loss = 0.017
epoch = 20, loss = 0.014
epoch = 30, loss = 0.012
epoch = 40, loss = 0.012


Accuracy = 0.510


## 数据预处理

## 权重初始化

## 激活函数

## 正则化

## 批归一化

## 参数更新策略

## 学习率调整

## 迁移学习

## 